# Generazione Playlist personalizzata in base all'umore

In [1]:
import os
import collections
from pathlib import Path
import pandas as pd
import numpy as np


# audio lib
import librosa
import librosa.display
from librosa.util import fix_length

import IPython.display as ipd
from PIL import Image

Si importa il file .csv contenente le informazioni prodotte dal modello MER

In [2]:
df = pd.read_csv('../input/tesi-finetuning/finetuned.csv')

# dati relativi alla creazione playlist, personalizzabili dall'utente
playlist_length = 10
starting_mood = 'A-V-'
expected_mood = 'A+V+'

L'algoritmo ideato funziona nel seguente modo:
* Si parte da una stima di 100 per il mood iniziale e di 0 per il mood da raggiungere
* Si calcola quanto si dovranno shiftare i due mood ogni iterazione, incrementando il target e decrescendo quello di partenza
* Si trovano le canzoni che hanno il mood di partenza e quello target più vicini ai valori ottenuti.

Es. playlist di 5 canzoni:
* 1° iterazione: valore_partenza = 100, valore_target = 0;
* 2° iterazione: valore_partenza = 75, valore_target = 25;
* 3° iterazione: valore_partenza = 50, valore_target = 50;
* 4° iterazione: valore_partenza = 25, valore_target = 75;
* 5° iterazione: valore_partenza = 0, valore_target = 100;

Se l'utente invece non vuole cambiare umore si prendono le canzoni con stima di quell'emozione più alta.

In [3]:
# funzione che prende in input le colonne relative alla stima del mood e il valore atteso a cui esse devono avvicinarsi
def find_nearest(array_starting, value_starting, array_target, value_target):
    
    # gli array contengono la differenza tra il valore reale e quello atteso
    starting = np.abs(array_starting - value_starting)
    target = np.abs(array_target - value_target)
    
    # si sommano i valori degli array
    nearest_array = starting + target
    
    # l'elemento minore sarà quello con i due mood combinati che più si avvicinano all'obiettivo
    return nearest_array.argmin()

In [4]:
def show_playlist(title_array):
    
    original_df = pd.read_csv('../input/tesi-finetuning/finetuned.csv')
    
    del original_df['Max_Overall_Emotion']
    del original_df['True_Emotion']
    del original_df['Wrong_Inferences']
    del original_df['diff_abs']
    
    my_playlist = original_df.iloc[:0,:].copy()
    for i in title_array:
        my_playlist = my_playlist.append(original_df.loc[original_df['ID'] == i], ignore_index=True)
    
    return my_playlist

In [5]:
song_list = []

if starting_mood != expected_mood: 
    # caso nel quale l'utente vuole shiftare mood
    jump = (100/(playlist_length-1))
    starting_value = 100
    expected_value = 0


    for i in range(playlist_length):

        idx = find_nearest(df[starting_mood].to_numpy(), starting_value, df[expected_mood].to_numpy(), expected_value)
        song_list.append(df.iloc[idx]['ID'])
        df = df.drop(idx)
        df = df.reset_index(drop=True)
        starting_value = starting_value - jump
        expected_value = expected_value + jump
        
else: # il mood dell'utente non deve cambiare
    df = df.sort_values(by=[starting_mood])
    df = df[len(df)-playlist_length:]
    song_list = df['ID'].to_numpy()


In [6]:
show_playlist(song_list)

,ID,A-V-,A-V+,A+V-,A+V+
0,Nirvana - All Apologies,75.0,12.5,0.0,12.5
1,Radiohead - Fake Plastic Trees,58.3,25.0,0.0,16.7
2,Muse - Feeling Good,50.0,12.5,12.5,25.0
3,David Bowie - Space Oddity,38.5,30.8,0.0,30.8
4,Machine Gun Kelly - Home,33.3,0.0,22.2,44.4
5,System Of A Down - Lonely Day,16.7,33.3,0.0,50.0
6,twenty one pilots - Ride,22.2,0.0,11.1,66.7
7,Soundgarden - Black Hole Sun,23.1,7.7,0.0,69.2
8,Rino Gaetano - Ma il cielo sempre pi blu,5.0,5.0,0.0,90.0
9,The Rolling Stones - Sympathy For The Devil,0.0,6.7,0.0,93.3


## Playlist generata:

In [7]:
for song in song_list:
    song_path = '../input/my-playlist/Playlist/' + song + '.mp3'
    print("Riproducendo:"+ song_path)
    # ipd.display(ipd.Audio(song_path)) 

Riproducendo:../input/my-playlist/Playlist/Nirvana - All Apologies.mp3
Riproducendo:../input/my-playlist/Playlist/Radiohead - Fake Plastic Trees.mp3
Riproducendo:../input/my-playlist/Playlist/Muse - Feeling Good.mp3
Riproducendo:../input/my-playlist/Playlist/David Bowie - Space Oddity.mp3
Riproducendo:../input/my-playlist/Playlist/Machine Gun Kelly - Home.mp3
Riproducendo:../input/my-playlist/Playlist/System Of A Down - Lonely Day.mp3
Riproducendo:../input/my-playlist/Playlist/twenty one pilots - Ride.mp3
Riproducendo:../input/my-playlist/Playlist/Soundgarden - Black Hole Sun.mp3
Riproducendo:../input/my-playlist/Playlist/Rino Gaetano - Ma il cielo  sempre pi blu.mp3
Riproducendo:../input/my-playlist/Playlist/The Rolling Stones - Sympathy For The Devil.mp3
